# Introduction

There are 2 datasets that serperatly span 3 years, 2018, 2019, 2021* : 
<br>
NSDUH - National Survey on Drug Use and Health which contains data on mental health
<br>
Core Trends - Pew Research Core trends survey which contains data on social media use

*2020 was not included because the Core Trends survey for that year doesn't exist.

With this data we are trying to answer whether there is a correlation between Social Media use and negative Mental health outcomes


In [1]:
import pandas as pd
import numpy as np
from ml_analysis import to_numeric, predictUsageOfAgeGroups

# EDA

# Visual 1

# Machine Learning

In [2]:
nsduh2018 = pd.read_csv('datasets/January 3-10, 2018 - Core Trends Survey/January 3-10, 2018 - Core Trends Survey - CSV.csv')
cleaned = to_numeric(nsduh2018, 'intfreq')
pred = predictUsageOfAgeGroups(cleaned)

Accuracy: 0.32
Classification Report:
              precision    recall  f1-score   support

       18-24       0.45      0.43      0.44       101
       25-34       0.21      0.26      0.24       136
       35-44       0.20      0.05      0.09       129
       45-54       0.26      0.07      0.11       155
       55-64       0.23      0.10      0.14       157
         65+       0.36      0.81      0.50       215

    accuracy                           0.32       893
   macro avg       0.29      0.29      0.25       893
weighted avg       0.28      0.32      0.26       893

Confusion Matrix:
[[ 43  33   1   1   1  22]
 [ 27  36  10   6  12  45]
 [ 12  32   7   9   9  60]
 [  4  30   9  11  18  83]
 [  4  22   5   8  15 103]
 [  6  15   3   7  10 174]]


After feeding the 2018 Core Trends data frame to the SVM model, we get a very low accuracy number. We're not entirely sure why this could be the case, but due to these results we plan on using Recursive Feature Elimination to help with finding the right columns to feed the model. For the results above, we are feeding it the internet frequency, the age, and which social medias they specifically use. There could be other responses that could help like, income, race, and marital status which are in the data.
This data frame has good features such as how much said users use certain social medias, 5 being rarely and 1 being several times a day. Which is what we're trying to predict, which social media is used the most amongst age groups.

What we plan to do is merge all the years, 2018, 2019 and 2021 to give the model more data to have a better accuracy score. However, some of the newer tests added new questions so the combined data frame has to be merged together and cleaned for it to inputted properly.